In [32]:
import pyarrow.parquet as pq
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from pathlib import Path

In [33]:
%reload_ext autoreload
%autoreload 2
#ignore warnings.
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

#imports
import pandas as pd
import os, sys
import importlib
import yaml, json
from pathlib import Path

#path append
sys.path.append(os.path.join(Path.cwd(), '../modules'))
import Evaluate, Helper, Present, Score, Synthetic


In [34]:
#multiple prediction file load still works, just loaded a single file because too many things are being displayed otherwise.
config_file = '../config/config.yaml'
prediction_config_files = ['../config/predictions/pred1.yaml']
c_r, c_e, c_gen, c_aws, c_visual, predictions=Helper.load_configuration(config_file, prediction_config_files)

In [35]:

cwd_dir = Path.cwd() #For running locally
base_dir = cwd_dir.parent
data_dir= base_dir / 'data' /'simulated_health_data' / 'data' 


Let's provide some examples of working with partitioned parquet files. 

In [36]:
df=pd.read_csv(data_dir / 'withtimeseries-10-100-10-5-2-True-10-5-2-True-2-101-5-0-0.csv')
df

,person_id,yyyymm,cad0,cad1,cad2,cad3,cad4,cad5,cad6,cad7,...,dv4,dv5,dv6,dv7,dv8,dv9,target_bin,target_class,target,index_c
0,1000,201701,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
1,1000,201702,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
2,1000,201703,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
3,1000,201704,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
4,1000,201705,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215995,9999,201808,1,1,1,0,0,0,0,0,...,0.487144,-0.686401,-0.218947,0.273039,0.011350,-0.370388,0,0,-27.788632,99
215996,9999,201809,1,1,1,0,0,0,0,0,...,0.487144,-0.686401,-0.218947,0.273039,0.011350,-0.370388,0,0,-27.788632,99
215997,9999,201810,1,1,1,0,0,0,0,0,...,0.487144,-0.686401,-0.218947,0.273039,0.011350,-0.370388,0,0,-27.788632,99
215998,9999,201811,1,1,1,0,0,0,0,0,...,0.487144,-0.686401,-0.218947,0.273039,0.011350,-0.370388,0,0,-27.788632,99


In [37]:
#Convert dataframe to table. 
table = pa.Table.from_pandas(df, preserve_index=False)

In [38]:
#write parquet file.
pq.write_table(table, data_dir /'withtimeseries-10-100-10-5-2-True-10-5-2-True-2-101-5-0-0.parquet')

#read parquet file.
parquet_file = pq.ParquetFile( data_dir /'withtimeseries-10-100-10-5-2-True-10-5-2-True-2-101-5-0-0.parquet')
parquet_file.metadata



  created_by: parquet-cpp version 1.5.1-SNAPSHOT
  num_columns: 27
  num_rows: 216000
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 11950

In [39]:
#read the schema
parquet_file.schema

person_id: INT64
yyyymm: INT64
cad0: INT64
cad1: INT64
cad2: INT64
cad3: INT64
cad4: INT64
cad5: INT64
cad6: INT64
cad7: INT64
cad8: INT64
cad9: INT64
cadre: INT64
dv0: DOUBLE
dv1: DOUBLE
dv2: DOUBLE
dv3: DOUBLE
dv4: DOUBLE
dv5: DOUBLE
dv6: DOUBLE
dv7: DOUBLE
dv8: DOUBLE
dv9: DOUBLE
target_bin: INT64
target_class: INT64
target: DOUBLE
index_c: INT64
 

In [40]:
#write to dataset without partition.  This will still chunk the file in to 2 parts. 
pq.write_to_dataset(table, root_path= str(data_dir / 'dataset_new'))

In [41]:
dataset = pq.ParquetDataset(str(data_dir / 'dataset_new'))
dataset_table = dataset.read()
dataset_table.schema


person_id: int64
yyyymm: int64
cad0: int64
cad1: int64
cad2: int64
cad3: int64
cad4: int64
cad5: int64
cad6: int64
cad7: int64
cad8: int64
cad9: int64
cadre: int64
dv0: double
dv1: double
dv2: double
dv3: double
dv4: double
dv5: double
dv6: double
dv7: double
dv8: double
dv9: double
target_bin: int64
target_class: int64
target: double
index_c: int64
metadata
--------
{b'pandas': b'{"index_columns": [], "column_indexes": [], "columns": [{"name":'
            b' "person_id", "field_name": "person_id", "pandas_type": "int64",'
            b' "numpy_type": "int64", "metadata": null}, {"name": "yyyymm", "f'
            b'ield_name": "yyyymm", "pandas_type": "int64", "numpy_type": "int'
            b'64", "metadata": null}, {"name": "cad0", "field_name": "cad0", "'
            b'pandas_type": "int64", "numpy_type": "int64", "metadata": null},'
            b' {"name": "cad1", "field_name": "cad1", "pandas_type": "int64", '
            b'"numpy_type": "int64", "metadata": null}, {"name": "cad2

# A partitioned dataset will let us read only 1 month, which will be optimal for quick modeling. 

In [42]:
#Create a partioned parque file. 
pq.write_to_dataset(table, root_path= str(data_dir / 'dataset_part'), partition_cols=['yyyymm'])

In [51]:
dataset = ds.dataset(str(data_dir / 'dataset_part'), format="parquet", partitioning=['yyyymm'])
dataset.to_table().to_pandas() 

,person_id,cad0,cad1,cad2,cad3,cad4,cad5,cad6,cad7,cad8,...,dv5,dv6,dv7,dv8,dv9,target_bin,target_class,target,index_c,yyyymm
0,1000,1,1,1,1,1,1,1,1,0,...,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0,yyyymm=201701
1,1001,1,1,1,1,1,1,1,1,0,...,0.955057,0.190794,1.978757,2.605967,0.683509,0,0,-84.698542,1,yyyymm=201701
2,1002,1,1,1,1,1,1,1,1,0,...,0.390528,0.166905,0.184502,0.807706,0.072960,0,0,-0.915500,2,yyyymm=201701
3,1003,1,1,1,1,1,1,1,1,0,...,0.484752,-0.116773,1.901755,0.238127,1.996652,0,0,-51.293606,3,yyyymm=201701
4,1004,1,1,1,1,1,1,1,1,0,...,-0.156598,-0.031579,0.649826,2.154846,-0.610259,0,0,-62.943629,4,yyyymm=201701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431995,9995,1,1,1,0,0,0,0,0,0,...,0.389736,0.389634,-0.012625,0.161977,1.499623,0,0,-5.806354,95,yyyymm=201812
431996,9996,1,1,1,0,0,0,0,0,0,...,-0.265327,-0.424594,1.868926,1.437781,-1.389828,0,0,0.429037,96,yyyymm=201812
431997,9997,1,1,1,0,0,0,0,0,0,...,0.190861,0.432215,-0.731353,-0.075012,2.529188,0,0,-153.759155,97,yyyymm=201812
431998,9998,1,1,1,0,0,0,0,0,0,...,-0.903090,-0.713788,-0.805162,1.179985,-0.080986,0,0,-182.604320,98,yyyymm=201812


In [61]:
dataset = ds.dataset(str(data_dir / 'dataset_part' / 'yyyymm=201701'), format="parquet")
dataset.to_table().to_pandas() 

,person_id,cad0,cad1,cad2,cad3,cad4,cad5,cad6,cad7,cad8,...,dv4,dv5,dv6,dv7,dv8,dv9,target_bin,target_class,target,index_c
0,1000,1,1,1,1,1,1,1,1,0,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
1,1001,1,1,1,1,1,1,1,1,0,...,-0.933237,0.955057,0.190794,1.978757,2.605967,0.683509,0,0,-84.698542,1
2,1002,1,1,1,1,1,1,1,1,0,...,-0.134841,0.390528,0.166905,0.184502,0.807706,0.072960,0,0,-0.915500,2
3,1003,1,1,1,1,1,1,1,1,0,...,-0.943406,0.484752,-0.116773,1.901755,0.238127,1.996652,0,0,-51.293606,3
4,1004,1,1,1,1,1,1,1,1,0,...,1.025984,-0.156598,-0.031579,0.649826,2.154846,-0.610259,0,0,-62.943629,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,9995,1,1,1,0,0,0,0,0,0,...,-0.814544,0.389736,0.389634,-0.012625,0.161977,1.499623,0,0,-5.806354,95
17996,9996,1,1,1,0,0,0,0,0,0,...,-0.308765,-0.265327,-0.424594,1.868926,1.437781,-1.389828,0,0,0.429037,96
17997,9997,1,1,1,0,0,0,0,0,0,...,-0.121841,0.190861,0.432215,-0.731353,-0.075012,2.529188,0,0,-153.759155,97
17998,9998,1,1,1,0,0,0,0,0,0,...,-0.252366,-0.903090,-0.713788,-0.805162,1.179985,-0.080986,0,0,-182.604320,98


# Can't get this to work and not sure why.
Here what I'm trying to do is read

https://arrow.apache.org/docs/python/dataset.html#filtering-data 

In [58]:
dataset = ds.dataset(str(data_dir / 'dataset_part'), format="parquet", partitioning=['yyyymm'])
dataset.to_table(filter=ds.field('yyyymm') >= 201801).to_pandas()

,person_id,cad0,cad1,cad2,cad3,cad4,cad5,cad6,cad7,cad8,...,dv5,dv6,dv7,dv8,dv9,target_bin,target_class,target,index_c,yyyymm
0,1000,1,1,1,1,1,1,1,1,0,...,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0,yyyymm=201701
1,1001,1,1,1,1,1,1,1,1,0,...,0.955057,0.190794,1.978757,2.605967,0.683509,0,0,-84.698542,1,yyyymm=201701
2,1002,1,1,1,1,1,1,1,1,0,...,0.390528,0.166905,0.184502,0.807706,0.072960,0,0,-0.915500,2,yyyymm=201701
3,1003,1,1,1,1,1,1,1,1,0,...,0.484752,-0.116773,1.901755,0.238127,1.996652,0,0,-51.293606,3,yyyymm=201701
4,1004,1,1,1,1,1,1,1,1,0,...,-0.156598,-0.031579,0.649826,2.154846,-0.610259,0,0,-62.943629,4,yyyymm=201701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431995,9995,1,1,1,0,0,0,0,0,0,...,0.389736,0.389634,-0.012625,0.161977,1.499623,0,0,-5.806354,95,yyyymm=201812
431996,9996,1,1,1,0,0,0,0,0,0,...,-0.265327,-0.424594,1.868926,1.437781,-1.389828,0,0,0.429037,96,yyyymm=201812
431997,9997,1,1,1,0,0,0,0,0,0,...,0.190861,0.432215,-0.731353,-0.075012,2.529188,0,0,-153.759155,97,yyyymm=201812
431998,9998,1,1,1,0,0,0,0,0,0,...,-0.903090,-0.713788,-0.805162,1.179985,-0.080986,0,0,-182.604320,98,yyyymm=201812


In [59]:
dataset = ds.dataset(str(data_dir / 'dataset_part'), format="parquet", partitioning=['yyyymm'])
dataset.to_table(filter=ds.field('yyyymm') == 201801).to_pandas()

,person_id,cad0,cad1,cad2,cad3,cad4,cad5,cad6,cad7,cad8,...,dv5,dv6,dv7,dv8,dv9,target_bin,target_class,target,index_c,yyyymm
